# Selenium Usage

反爬設置：若需避免反爬，可以加上自訂的 User-Agent 或模擬人類行為（如隨機點擊、等待）

1. 設置自訂的 User-Agent

    許多網站會檢測 User-Agent，如果發現是無頭瀏覽器，可能會拒絕訪問。可以通過 chrome_options 設置自訂的 User-Agent。

        from selenium.webdriver.chrome.options import Options

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.198 Safari/537.36")


2. 禁用 Selenium 的特徵標記
    某些網站會檢測瀏覽器中的特徵標記（如 navigator.webdriver），以此識別 Selenium。可以通過修改 JavaScript 屬性來隱藏這些標記。

        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
            Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
            """
        })


3. 隨機延遲操作 
    爬取時添加隨機延遲來模仿人類操作，而不是直接快速執行每個動作。
        import time
        import random
        def random_delay():
            time.sleep(random.uniform(2, 5))  # 隨機延遲 2 到 5 秒
        使用時：
        random_delay()
        driver.get("https://example.com")
        random_delay()


4. 設置瀏覽器窗口大小
    某些網站會檢測瀏覽器窗口大小，無頭模式默認使用最小窗口，可以手動設置為普通大小。
        chrome_options.add_argument("window-size=1920,1080")

5. 禁用無頭模式標記
    某些網站能識別到無頭瀏覽器，可以通過禁用一些無頭模式特徵來隱藏。
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")

6. 模擬鼠標和鍵盤操作 通過 Selenium 的 ActionChains 模組模擬滑鼠移動、點擊和鍵盤輸入。

        from selenium.webdriver.common.action_chains import ActionChains
        from selenium.webdriver.common.keys import Keys

        action = ActionChains(driver)
        element = driver.find_element("name", "q")  # 找到 Google 搜索框
        action.move_to_element(element).click().send_keys("Selenium").send_keys(Keys.RETURN).perform()


7. 使用代理 IP
    使用代理 IP 可以減少目標網站對於同一 IP 的訪問限制。
        
        chrome_options.add_argument("--proxy-server=http://your-proxy-server:port")
        
        若需要動態代理，建議使用服務如 ProxyMesh 或 Bright Data。

8. 多用戶代理切換
    通過腳本自動切換不同的代理 IP，減少單一 IP 的檢測風險。

# Install packages

In [1]:
# 安裝所需套件
!pip install selenium webdriver-manager
!pip install fake-useragent -U



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Attempting uninstall: fake-useragent
    Found existing installation: fake-useragent 2.0.3
    Uninstalling fake-useragent-2.0.3:
      Successfully uninstalled fake-useragent-2.0.3



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# 安裝 Google Chrome
# !apt-get update
# !apt-get install -y wget
# !wget -q -O google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# !dpkg -i google-chrome-stable_current_amd64.deb
# !apt-get -f install -y


In [3]:

# 實作程式碼
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import random


In [4]:
# 隨機延遲 (random_delay)
# 模擬人類操作的速度，避免連續的快速請求引起網站注意。

# 自訂 User-Agent
# 模仿真實的瀏覽器標識，避免被識別為自動化工具。

# 隱藏 Selenium 的特徵
# 使用 JavaScript 修改 navigator.webdriver，防止網站檢測到 Selenium 的執行環境。

# 設置窗口大小
# 無頭模式下默認窗口大小可能會暴露機器特徵，設置常見的分辨率如 1920x1080。

# 模擬輸入與提交
# 使用 Selenium 的 send_keys 和 submit 進行互動操作，模擬用戶行為。

#-------------------
# 隨機延遲函數
def random_delay():
    time.sleep(random.uniform(5, 15))  # 隨機延遲 2 到 5 秒

# 設定 Chrome Options
chrome_options = Options()
chrome_options.add_argument("--headless")  # 無頭模式 不顯示瀏覽器界面
chrome_options.add_argument("--no-sandbox") # 避免沙盒
chrome_options.add_argument("--incognito") # 無痕模式
chrome_options.add_argument("--disable-dev-shm-usage") # 避免記憶體不足
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 隱藏無頭模式特徵
chrome_options.add_argument("window-size=1920,1080")  # 模擬普通瀏覽器視窗大小
# 禁止印出log、隱藏受自動化軟體控制之資訊
# 禁用Chrome的自動化擴充功能
# 可以避免某些網站檢測到自動化測試而阻擋訪問
# 提高爬蟲的穩定性和真實性
# 這些設定的主要目的是:

# 讓自動化程式更像真實用戶在使用瀏覽器
# 避免被網站檢測出是自動化工具
# 減少不必要的日誌輸出
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging', 'enable-automation'])
chrome_options.add_experimental_option('useAutomationExtension', False)

# 許多網站會檢測 User-Agent，如果發現是無頭瀏覽器，可能會拒絕訪問。可以通過 chrome_options 設置自訂的 User-Agent。
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.198 Safari/537.36"
)  # 設定自訂 User-Agent

# 自動下載與配置 ChromeDriver 它會根據你的 Chrome 瀏覽器版本自動匹配合適的 ChromeDriver，免除手動下載與管理
#service = Service(ChromeDriverManager().install())
# 啟動 Selenium WebDriver
#driver = webdriver.Chrome(service=service, options=chrome_options)

# 啟動 Selenium WebDriver
try:
    service = Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(
        service=service,
        options=chrome_options
    )
except Exception as e:
    print(f"Error initializing Chrome browser: {e}")
    raise # Keep this to ensure program stops on critical error


# 禁用 Selenium 的特徵標記
# 隱藏 Selenium 特徵
browser.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {
        "source": """
        Object.defineProperty(navigator, 'webdriver', { get: () => undefined });
        """
    },
)


{'identifier': '2'}

In [5]:
def crawl_some_information():
    try:
        # 訪問維基百科頁面
        browser.get("https://zh.wikipedia.org/wiki/Python")
        print("Page Title:", browser.title)

        # 隨機延遲，模擬人類行為
        random_delay()

        # 取得頁面中的第一段介紹文字
        intro_text = browser.find_element("css selector", "#mw-content-text p:not(.mw-empty-elt)").text
        print("\n介紹文字:")
        print(intro_text)

        # 取得目錄標題
        toc_items = browser.find_elements("css selector", "#toc ul li .toctext")
        print("\n目錄項目:")
        for item in toc_items[:5]:  # 只印出前5個目錄項目
            print("- " + item.text)

        random_delay()
        
    except Exception as e:
        print(f"發生錯誤: {e}")
    finally:
        # 關閉瀏覽器
        browser.quit()
        print("瀏覽器已關閉。")

crawl_some_information()

Page Title: Python - 維基百科，自由的百科全書

介紹文字:
Python（英式發音：/ˈpaɪθən/；美式發音：/ˈpaɪθɑːn/），是一種廣泛使用的直譯式、進階和通用的程式語言。Python支援多種程式設計範式，包括結構化、程序式、反射式、物件導向和函數式程式設計。它擁有動態型別系統和垃圾回收功能，能夠自動管理主記憶體使用，並且其本身擁有一個巨大而廣泛的標準庫。它的語言結構以及物件導向的方法，旨在幫助程式設計師為小型的和大型的專案編寫邏輯清晰的程式碼。

目錄項目:
瀏覽器已關閉。
